# EEG Machine Learning Pipeline

## TO DO - To Review

Projects:
- Pickle: https://www.perplexity.ai/search/in-a-jupyter-notebook-i-have-c-0LbAAH9ITFGfcPYaWlrt6Q

In [1]:
# General imports
import os
import gc
from datetime import datetime
from pprint import pprint

import math
import numpy as np
import pandas as pd

# Plots
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
# A utility function to establish relative paths for a given folder
def get_folder_path(folder_name):
    project_root = os.path.dirname(os.getcwd())
    folder_path = os.path.join(project_root, folder_name)
    if not os.path.isdir(folder_path):
        raise FileNotFoundError(f'Directory not found: {folder_path}')  
     
    return folder_path

# Utility function to check for the existence of a file in a given directory
def get_file_path(folder, file_name):
    file_path = os.path.join(folder, file_name)
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f'File not found: {file_path}')
    return file_path

In [3]:
def load_dataframe(file_path):

    # Validate
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f'File not found: {file_path}')

    try:
        if not os.path.isfile(file_path):
            raise FileNotFoundError(f'File not found: {file_path}')
        
        df = pd.read_pickle(file_path)
        print(f"DataFrame loaded: shape {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading DataFrame: {e}")
        return None

In [4]:
study_features_folder = get_folder_path('Data/EEG_Datasets_Processed/ds004584-1.0.0_20250615')
study_features_file = 'study_features_df.pkl'

eeg_study_features_file_path = get_file_path(study_features_folder, study_features_file)

In [5]:
test = load_dataframe(eeg_study_features_file_path)

DataFrame loaded: shape (149, 2248)
